Web Scraping - Scraping the data from a website.

BeautifulSoap is a class from bs4 python library (library used for web scraping), request is use to get the info via website link.

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

'url' is the website link, headers is used to make the website feel that server is giving request to access the data so it is not interpreted as a bot request.

In [25]:
url = 'https://www.ambitionbox.com/list-of-companies?page=1'
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0 Safari/537.36"}
r = requests.get(url, headers=headers, verify=False)
print('STATUS', r.status_code)
webpage = r.text

STATUS 200


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ambitionbox.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


It converts raw HTML into a structured object you can navigate like a tree.
Be

In [14]:
soup=BeautifulSoup(webpage,'lxml')

It prints a formatted, indented version of the parsed HTML tree.

In [15]:
# print(soup.prettify())

To find all the h1 heading and print out the content using soup.find_all('h1').

In [16]:
clean = soup.find_all('h1')[0].get_text(separator=' ', strip=True).replace('\xa0',' ')
print(clean)

Top Companies in INDIA


TO get all content with all h2 headings.

In [17]:
for i in soup.find_all('h2'):
  print(i.text.strip())

Companies in India
TCS
Accenture
Wipro
Cognizant
Capgemini
HDFC Bank
Infosys
ICICI Bank
HCLTech
Tech Mahindra
Genpact
Teleperformance
Jio
Axis Bank
Concentrix Corporation
Amazon
iEnergizer
Reliance Retail
LTIMindtree
IBM
Popular Collections by Industries
Popular Collections by Cities
Popular Collections by Roles


In [18]:
for i in soup.find_all('p'):
  print(i.text.strip())

AmbitionBox
About Company
Discover best places to work
Compare & find best workplace
Bring your workplace to life
Highlight your company's perks
Read reviews for 6L+ companies
Rate your former or current company
Discover salaries for 6L+ companies
Calculate your take home salary
Check your market value
Help other jobseekers
Check your gratuity amount
Check how much of your HRA is tax-free
Check your salary hike
Read interviews for 40K+ companies
Contribute your interview questions
AmbitionBox Employee Choice Awards - 5th Edition
AmbitionBox Employee Choice Awards - 4th Edition
AmbitionBox Employee Choice Awards - 3rd Edition
Popular
AmbitionBox Award Winner'25
AmbitionBox Award Winner'25
AmbitionBox Award Winner'25
AmbitionBox Award Winner'25
AmbitionBox Award Winner'25
AmbitionBox Award Winner'25
AmbitionBox Award Winner'25
Side-by-side comparison to make informed career
										decisions
Trusted by over
									1.5 Crore
									job seekers to find their right fit company
Reviews

To get the rating.

In [19]:
ratings = soup.find_all('div', class_='rating_text')

for r in ratings:
    print(r.text.strip())


3.3
3.7
3.6
3.6
3.7
3.8
3.5
4.0
3.4
3.4
3.6
3.8
4.4
3.6
3.6
3.9
4.6
3.9
3.6
3.9


To find all the div which contain the info about the company

In [20]:

company=soup.find_all('div',class_='companyCardWrapper')
len(company)
     

20

In [21]:
name = []
rating = []
reviews = []
salaries = []
cities = []
c_type = []

for i in company:

    name.append(i.find('h2').text.strip())
    rating.append(i.find('div', class_='rating_text').text.strip())
    
    actions = i.find_all('a', class_='companyCardWrapper__ActionWrapper')
    reviews.append(actions[0].text.strip())
    salaries.append(actions[1].text.strip())
    
    info = i.find_all('span', class_='companyCardWrapper__interLinking')[0].text.strip()
    
    c_type, rest = info.split('|')
    city = rest.split('+')[0].strip()
    
    cities.append(city)

df = pd.DataFrame({
    'Name': name,
    'Rating': rating,
    'Company_type': c_type,
    'Reviews': reviews,
    'Salaries': salaries,
    'Cities': cities,
})

print(df)
df.to_csv('company_data.csv', index=False)


                      Name Rating               Company_type        Reviews  \
0                      TCS    3.3  IT Services & Consulting    1.1L Reviews   
1                Accenture    3.7  IT Services & Consulting     72k Reviews   
2                    Wipro    3.6  IT Services & Consulting   63.9k Reviews   
3                Cognizant    3.6  IT Services & Consulting   60.3k Reviews   
4                Capgemini    3.7  IT Services & Consulting   51.9k Reviews   
5                HDFC Bank    3.8  IT Services & Consulting   51.1k Reviews   
6                  Infosys    3.5  IT Services & Consulting   47.7k Reviews   
7               ICICI Bank    4.0  IT Services & Consulting   45.2k Reviews   
8                  HCLTech    3.4  IT Services & Consulting   44.9k Reviews   
9            Tech Mahindra    3.4  IT Services & Consulting   42.6k Reviews   
10                 Genpact    3.6  IT Services & Consulting   41.3k Reviews   
11         Teleperformance    3.8  IT Services & Con

In [28]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

df = pd.DataFrame()
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0 Safari/537.36"}

for j in range(499, 500):
    url = f'https://www.ambitionbox.com/list-of-companies?page={j}'
    
    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()  # Raise exception for bad status codes
        
        print(f"Page {j}: Status {r.status_code}")
        
    except requests.exceptions.RequestException as e:
        print(f"Page {j}: Error - {e}")
        break
    
    soup = BeautifulSoup(r.text, 'lxml')
    company = soup.find_all('div', class_='companyCardWrapper')
    
    # Stop if no companies found (reached end of pages)
    if not company:
        print(f"Page {j}: No companies found - stopping")
        break

    name = []
    rating = []
    reviews = []
    salaries = []
    cities = []
    ctype_list = []

    for i in company:
        try:
            company_name = i.find('h2').text.strip()
            company_rating = i.find('div', class_='rating_text').text.strip()

            actions = i.find_all('a', class_='companyCardWrapper__ActionWrapper')
            company_reviews = actions[0].text.strip()
            company_salaries = actions[1].text.strip()

            info = i.find_all('span', class_='companyCardWrapper__interLinking')[0].text.strip()

            ctype, sep, rest = info.partition('|')

            if sep:
                city = rest.split('+')[0].strip()
            else:
                city = None

            # Only append if all data was extracted successfully
            name.append(company_name)
            rating.append(company_rating)
            reviews.append(company_reviews)
            salaries.append(company_salaries)
            ctype_list.append(ctype.strip())
            cities.append(city)
        
        except (IndexError, AttributeError) as e:
            print(f"Page {j}: Error parsing company - {e}")
            continue

    d = pd.DataFrame({
        'Name': name,
        'Rating': rating,
        'Company_type': ctype_list,
        'Reviews': reviews,
        'Salaries': salaries,
        'Cities': cities,
    })

    df = pd.concat([df, d], ignore_index=True)
    print(f"Page {j}: Scraped {len(company)} companies. Total: {len(df)}")
    
    # Be respectful to the server - add delay between requests
    time.sleep(2)

print(f"\nTotal companies scraped: {len(df)}")
df.to_csv('company_data.csv', index=False)
print("Data saved to company_data.csv")

Page 499: Status 200
Page 499: Scraped 20 companies. Total: 20

Total companies scraped: 20
Data saved to company_data.csv


Making a dataset from website, finding the headings that are needed to be extracted and than inspecting the website and finding the data and converting it to a dataframe.
Started of with 'url' (only one page for now) then giving 'headers' to avoid 403 error. Then used 'requests' to get the webpage and stored it in a variable 'webpage'. Using BeautifulSoup to parse the webpage and stored it in a variable 'soup'. Then used 'find_all' to find all the divs with class 'companyCardWrapper' and stored it in a variable 'company'. Then used a for loop to iterate through each company card and extract the required information and stored it in respective lists. Finally, created a dataframe using pandas and printed it.

Basic fundamental for web scraping is to 'inspect' the website and hover down to the elements which you act upon and then use BeautifulSoup to extract that data. 